In [1]:
import datetime
import os
import re
import subprocess
import time

In [2]:
def listMacAddresses():
    process = subprocess.Popen('airport -s', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process.wait()
    processOutput = process.stdout.read().decode('utf-8')

    lines = list(filter(lambda x: len(x) > 0, processOutput.split('\n')))
    header = lines[0]
    reHeader = re.compile('\s+SSID')
    networkNameLength = len(reHeader.search(header).group())
    parsedValues = [parseAirportLine(line, networkNameLength) for line in lines]
    
    return parsedValues

In [3]:
def parseAirportLine(line, networkNameLength):
    if len(line) == 0:
        return

    name = re.sub('\s+', ' ', line[:networkNameLength]).strip()
    regex = re.compile('\s+')
    bssid = regex.split(line[networkNameLength:])[1]

    return {
        'name': name,
        'bssid': bssid,
    }

In [4]:
fileName = './logs.txt'
readingMode = 'a'
if not os.path.exists(fileName):
    readingMode = 'w+'

while True:
    with open(fileName, readingMode) as logFile:
        addresses = listMacAddresses()
        currentTime = str(datetime.datetime.now())
        logs = list(map(lambda x: ';'.join([currentTime] + list(x.values()) + ['\n']), addresses[1:]))
        print('{} - Writting logs...'.format(currentTime))
        logFile.write(''.join(logs))
        logFile.close()

    time.sleep(10)

2018-02-13 17:04:26.922191 - Writting logs...
2018-02-13 17:04:45.252867 - Writting logs...


KeyboardInterrupt: 